<a href="https://colab.research.google.com/github/viniciusrpb/cloud_image_classification/blob/main/cloud_classification_vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

In [2]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 14.1 MB/s 


In [3]:
!cp -r "/content/drive/My Drive/img_satelite/classificacao/CCSN/train" "training"
!cp -r "/content/drive/My Drive/img_satelite/classificacao/CCSN/val" "validation"
!cp -r "/content/drive/My Drive/img_satelite/classificacao/CCSN/test" "testing"

In [4]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,GlobalAveragePooling2D ,MaxPooling2D,Activation,Flatten,Conv2D,BatchNormalization,Dropout
from tensorflow.keras import layers
from tensorflow.keras import Model
import keras.applications
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [5]:
path_train = 'training'
path_validation = 'validation'
path_test = 'testing'

# Pretrained VGG16 model using ImageNet


Hyperparameters

In [6]:
batch_size = 16
epochs=70
dim = (224,224)

In [7]:
#from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16

pre_trained_model = VGG16(input_shape=(224,224,3),include_top=False,pooling ='avg',weights='imagenet', classes=11)

for layer in pre_trained_model.layers:
  layer.trainable = False

58900480/58889256 [==============================] - 1s 0us/step


### Data augmentation

In [8]:
#train_datagen = ImageDataGenerator(rescale=1. / 255,brightness_range=[0.4,1.5],zoom_range=0.3,horizontal_flip=True, vertical_flip=True,width_shift_range=0.2, height_shift_range=0.2,rotation_range=30, fill_mode='nearest')

train_datagen = ImageDataGenerator(rescale=1. / 255)

validation_datagen = ImageDataGenerator(rescale=1. / 255)

test_datagen = ImageDataGenerator(rescale=1. / 255)


Read the images

In [9]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(path_train,
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical', 
                                                    target_size = (224,224))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  validation_datagen.flow_from_directory( path_validation,
                                                          batch_size  = batch_size,
                                                          class_mode  = 'categorical', 
                                                          target_size = (224,224))

test_generator =  test_datagen.flow_from_directory( path_test,
                                                          batch_size  = batch_size,
                                                          class_mode  = 'categorical', 
                                                          target_size = (224,224))

Found 1774 images belonging to 11 classes.
Found 250 images belonging to 11 classes.
Found 519 images belonging to 11 classes.


Hyper-parameters

In [10]:
number_of_neurons = 1024
number_of_classes = 11

Lets see some samples

In [11]:
model = Sequential()

model.add(pre_trained_model)
model.add(BatchNormalization())
#model.add(GlobalAveragePooling2D())
model.add(Flatten())

# 1st hidden layer
model.add(Dense(number_of_neurons,activation="relu"))
model.add(Dropout(0.2))
#model.add(BatchNormalization())

model.add(Dense(number_of_neurons//2,activation="relu"))
model.add(Dropout(0.2))
#model.add(BatchNormalization())


# 3rd hidden layer : 75 neurons
#model.add(Dense(number_of_neurons//4,activation="relu"))
#model.add(Dropout(0.5))
#model.add(BatchNormalization())

model.add(Dense(number_of_classes,activation="softmax"))

In [12]:
sgd = SGD(learning_rate= 0.01, momentum = 0.9, nesterov = False)

f1_score = tfa.metrics.F1Score(num_classes=number_of_classes, average='macro',threshold=0.5)

model.compile(optimizer = sgd, 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy',f1_score])



Callbacks definition to reduce chances of overfitting the model

In [13]:
fname = "weights-vgg16-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"

resnet_checkpoint = ModelCheckpoint(fname, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

resnet_early_stopping = EarlyStopping(monitor='loss', patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.05, patience=5, min_lr=0.000002)
callbacklist = [resnet_checkpoint,resnet_early_stopping,reduce_lr]

In [14]:
history_fine = model.fit(train_generator,
                         epochs=epochs,
                         batch_size=batch_size,
                         validation_data=validation_generator,
                         callbacks=callbacklist
                         )

Epoch 1/70
111/111 [==============================] - ETA: 0s - loss: 2.0214 - accuracy: 0.3044 - f1_score: 0.1643
Epoch 1: val_accuracy improved from -inf to 0.17200, saving model to weights-vgg16-improvement-01-0.17.hdf5
111/111 [==============================] - 1045s 9s/step - loss: 2.0214 - accuracy: 0.3044 - f1_score: 0.1643 - val_loss: 2.2402 - val_accuracy: 0.1720 - val_f1_score: 0.0000e+00 - lr: 0.0100
Epoch 2/70
111/111 [==============================] - ETA: 0s - loss: 1.6849 - accuracy: 0.4233 - f1_score: 0.2607
Epoch 2: val_accuracy improved from 0.17200 to 0.33600, saving model to weights-vgg16-improvement-02-0.34.hdf5
111/111 [==============================] - 1022s 9s/step - loss: 1.6849 - accuracy: 0.4233 - f1_score: 0.2607 - val_loss: 2.0083 - val_accuracy: 0.3360 - val_f1_score: 0.0154 - lr: 0.0100
Epoch 3/70
111/111 [==============================] - ETA: 0s - loss: 1.5074 - accuracy: 0.4729 - f1_score: 0.3577
Epoch 3: val_accuracy improved from 0.33600 to 0.36800, 

KeyboardInterrupt: ignored

In [ ]:
#f1 = history_fine.history['f1_score']
#val_f1 = history_fine.history['val_f1_score']

#plt.figure(figsize=(8, 8))
#plt.subplot(2, 1, 1)
#plt.plot(f1, label='Training F1-Score')
#plt.plot(val_f1, label='Validation F1-Score')
#plt.legend(loc='lower right')
#plt.ylabel('F1-Score')
#plt.ylim([min(plt.ylim()),1])
#plt.title('Training and Validation F1-Score')


In [ ]:
f1 = history_fine.history['accuracy']
val_f1 = history_fine.history['val_accuracy']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(f1, label='Training F1-Score')
plt.plot(val_f1, label='Validation F1-Score')
plt.legend(loc='lower right')
plt.ylabel('F1-Score')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation F1-Score')


In [ ]:
loss = history_fine.history['loss']
val_loss = history_fine.history['val_loss']

plt.plot(history_fine.history['loss'])
plt.plot(history_fine.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper left')
plt.show()

In [ ]:
score = model.evaluate(test_generator,batch_size=batch_size) 
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

In [ ]:
#y_prob = model.predict(test_generator,batch_size=batch_size)
#y_pred = np.argmax(y_prob,axis=1)

y_prob = model.predict(test_generator)
y_pred = np.argmax(y_prob,axis=1)
#print(pred, labels[np.argmax(pred)])
print(classification_report(test_generator.classes,y_pred))

In [ ]:
test_generator.classes

In [ ]:
y_pred

In [ ]:
import plotly.figure_factory as ff

y_true = test_generator.classes

z = confusion_matrix(y_true, y_pred)

x = list(test_generator.class_indices.keys())

y = list(test_generator.class_indices.keys())


In [ ]:
# change each element of z to type string for annotations
z_text = [[str(y) for y in x] for x in z]

# set up figure 
fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

# add title
fig.update_layout(title_text='<i><b>Confusion matrix</b></i>',
                  #xaxis = dict(title='x'),
                  #yaxis = dict(title='x')
                 )

# add custom xaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.15,
                        showarrow=False,
                        text="Predicted value",
                        xref="paper",
                        yref="paper"))

# add custom yaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.35,
                        y=0.5,
                        showarrow=False,
                        text="Real value",
                        textangle=-90,
                        xref="paper",
                        yref="paper"))

# adjust margins to make room for yaxis title
fig.update_layout(margin=dict(t=50, l=200))

# add colorbar
fig['data'][0]['showscale'] = True
fig.show()